### setup

In [1]:
%load_ext lab_black

In [2]:
#%pip install cartopy
#%pip install vega_datasets

In [3]:
import pandas as pd
import requests
import pageviewapi
import datetime as dt
from bs4 import BeautifulSoup
import altair as alt
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import geopandas as gpd
import us
from cartopy import crs as ccrs
from vega_datasets import data

In [4]:
import matplotlib

In [5]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [6]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="PU52xuCQowgE5BFprvdnpFXzd5Ql9ISOSLBk58eNR2ykPztw0yC5fpqLTfsZdhyR"
)

In [7]:
# scope = ['https://spreadsheets.google.com/feeds']
# credentials = ServiceAccountCredentials.from_json_keyfile_name('jupyter-integration-349314-25735d35924b.json', scope)
# gc = gspread.authorize(credentials)

### read data

In [8]:
# 1980-2022 Billion-Dollar Disasters Cost (in $Millions) By State (CPI-Adjusted)
state_df = pd.read_csv("data/raw/hurricanes/state-cost-data.csv", skiprows=1)

In [9]:
# Billion-Dollar Disasters By Year (CPI-Adjusted)
df = pd.read_csv("data/raw/hurricanes/state-freq-data.csv", skiprows=1).rename(
    columns={"state": "abbr"}
)

In [10]:
df.columns = df.columns.str.replace(" ", "_")

In [11]:
df.head()

,year,abbr,drought,flooding,freeze,severe_storm,tropical_cyclone,wildfire,winter_storm
0,1980,AK,0,0,0,0,0,0,0
1,1980,AL,1,0,0,0,0,0,0
2,1980,AR,1,1,0,0,0,0,0
3,1980,AZ,0,0,0,0,0,0,0
4,1980,CA,0,0,0,0,0,0,0


In [12]:
annual = df.groupby("year").agg(sum).reset_index()

In [13]:
col_list = list(annual)

col_list.remove("year")

In [14]:
annual["all_events"] = annual[col_list].sum(axis=1)

In [15]:
annual["year"] = annual["year"].astype(str)
annual["rollmean_hurricanes"] = annual["tropical_cyclone"].rolling(5).mean()
annual["rollmean_all"] = annual["all_events"].rolling(5).mean()

In [16]:
annual_long = annual.melt(
    id_vars="year", value_vars=["rollmean_hurricanes", "rollmean_all"]
)

In [17]:
alt.Chart(annual_long).mark_area().encode(
    x=alt.X("year:O"), y=alt.Y("value"), color=alt.Color("variable:N")
)

alt.Chart(...)

In [21]:
# dw.create_chart(
#    "billion dollar events",
#    chart_type="d3-area",
#    data=annual[["year", "rollmean_storms", "rollmean_all"]],
# )

chart_id = "7zCjK"

dw.add_data(
    chart_id=chart_id, data=annual[["year", "rollmean_hurricanes", "rollmean_all"]]
)

<Response [204]>

In [19]:
annual_slim = annual[["year", "all_events", "tropical_cyclone"]].melt(
    id_vars="year", value_vars=["all_events", "tropical_cyclone"]
)

In [20]:
alt.Chart(annual_slim).mark_area().encode(
    x=alt.X("year:O"), y=alt.Y("value"), color=alt.Color("variable:N")
)

alt.Chart(...)